# SparkDataFrames

https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.html

In [1]:
# Installing required packages
!pip install pyspark
!pip install findspark
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=0b231e033a290a24cc3591cd9b8359d0821ddf94c8d6a2ef6aed7767c9022a9a
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


## Imports & Load Data

In [2]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark import SparkContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col, avg , sum

import findspark

findspark.init()

df = pd.read_csv("cleaned_music_streaming.csv")

df.head()

,artist,track,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration,time_signature,genre
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,0.177348,0.0849,0.8990,134.071,234.596,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251.733,4,10
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109.667,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,0.177348,0.1220,0.5690,107.033,173.968,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229.960,4,10


## Create Spark Context and Sessoin

In [3]:
# Context
SparkContext = SparkContext()

# Session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
spark_df = spark.createDataFrame(df)

## PrintSchema & Show

In [5]:
spark_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- track: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: long (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration: double (nullable = true)
 |-- time_signature: long (nullable = true)
 |-- genre: long (nullable = true)



In [6]:
spark_df.show(5)

+--------------------+--------------------+----------+------------+------+----+--------+----+-----------+------------+------------------+--------+-------+-------+--------+--------------+-----+
|              artist|               track|popularity|danceability|energy| key|loudness|mode|speechiness|acousticness|  instrumentalness|liveness|valence|  tempo|duration|time_signature|genre|
+--------------------+--------------------+----------+------------+------+----+--------+----+-----------+------------+------------------+--------+-------+-------+--------+--------------+-----+
|          Bruno Mars|That's What I Lik...|      60.0|       0.854| 0.564| 1.0|  -4.964|   1|     0.0485|      0.0171|0.1773476204719195|  0.0849|  0.899|134.071| 234.596|             4|    5|
|              Boston|        Hitch a Ride|      54.0|       0.382| 0.814| 3.0|   -7.23|   1|     0.0406|      0.0011|           0.00401|   0.101|  0.569|116.454| 251.733|             4|   10|
|       The Raincoats|  No Side to 

## Temp View of Data

In [7]:
spark_df.createOrReplaceTempView("music_streaming")

## Spark operations

### Genre with the highest average popularity

In [8]:
avg_popularity_by_genre_df = spark_df.groupBy("genre").agg(avg("popularity").alias("avg_popularity"))
top_5_popular_genres = avg_popularity_by_genre_df.orderBy(col("avg_popularity").desc()).limit(5)

print("Top 5 genres with the highest average popularity:")
top_5_popular_genres.show()

Top 5 genres with the highest average popularity:
+-----+-----------------+
|genre|   avg_popularity|
+-----+-----------------+
|    4|56.77410851584384|
|    9|54.87524150977302|
|    5|51.07870208095271|
|   10|48.40947614489272|
|    1|45.86965588796474|
+-----+-----------------+



### Artists have recorded the most number of songs with a duration of more than 5 minutes

In [9]:
# Filter songs with a duration of more than 5 minutes
songs_gt_5min_df = spark_df.filter(col("duration") > 5)

# Group by artist and count the number of songs
artist_song_count_df = songs_gt_5min_df.groupBy("artist").count()

# Sort the result in descending order by the count of songs
artist_song_count_df = artist_song_count_df.orderBy(col("count").desc())

# Display the top artist(s) with the most number of songs
print("Top artist(s) with the most number of songs with duration > 5 minutes:")
artist_song_count_df.show(5)

Top artist(s) with the most number of songs with duration > 5 minutes:
+------------------+-----+
|            artist|count|
+------------------+-----+
|The Rolling Stones|   31|
|                U2|   27|
|         Metallica|   27|
|    The Black Keys|   22|
|           Nirvana|   22|
+------------------+-----+
only showing top 5 rows



### Number of songs are included in every Genre

In [10]:
# Group by genre and count the number of songs for each genre
genre_song_count_df = spark_df.groupBy("genre").count()

print("Number of songs included in every genre:")
genre_song_count_df.show()

Number of songs included in every genre:
+-----+-----+
|genre|count|
+-----+-----+
|    0|  586|
|    7|  465|
|    6| 2263|
|    9| 1828|
|    5| 1210|
|    1| 1268|
|   10| 4264|
|    3|  371|
|    8| 1704|
|    2| 1182|
|    4|  376|
+-----+-----+



### Artists dominated the charts

In [11]:
# Group by artist and calculate the total sum of popularity for each artist
artist_popularity_sum_df = spark_df.groupBy("artist").agg(sum("popularity").alias("total_popularity"))

# Sort the result in descending order by the total sum of popularity
dominant_artists_df = artist_popularity_sum_df.orderBy(col("total_popularity").desc())

# Display the top artist(s) with the highest total sum of popularity
print("Top artist(s) that dominated the charts based on total sum of popularity:")
dominant_artists_df.show(5)

Top artist(s) that dominated the charts based on total sum of popularity:
+------------------+------------------+
|            artist|  total_popularity|
+------------------+------------------+
|    Britney Spears| 2637.241221979766|
|   Backstreet Boys|            2615.0|
|The Rolling Stones|1838.3294319910071|
|         Metallica|            1710.0|
|                U2|            1648.0|
+------------------+------------------+
only showing top 5 rows



### Least 5 fun/not-boring songs that can be played at a party

In [12]:
fun_songs_df = spark_df.filter((col("energy") > 0.7) & (col("danceability") > 0.7))
top_fun_songs_df = fun_songs_df.orderBy(col("popularity").desc()).limit(5)

print("Recommended fun/not-boring songs for a party:")
top_fun_songs_df.select("artist", "track").show(truncate=False)


Recommended fun/not-boring songs for a party:
+---------------+------------------------+
|artist         |track                   |
+---------------+------------------------+
|Måneskin       |Beggin'                 |
|Doja Cat       |Kiss Me More (feat. SZA)|
|Ed Sheeran     |Bad Habits              |
|Doja Cat, SZA  |Kiss Me More (feat. SZA)|
|Los Legendarios|Fiel                    |
+---------------+------------------------+



# Spark ML

### Split the data into training and testing sets & Vector assembler

Vector assembler : A feature transformer that merges multiple columns into a vector column.

In [29]:
from pyspark.ml.classification import GBTClassifier, LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

train_df, test_df = spark_df.randomSplit([0.60, 0.40])

assembler = VectorAssembler(inputCols=['acousticness', 'danceability',
                                       'energy',
                                       'instrumentalness',
                                       'liveness',
                                       'valence',
                                       'loudness',
                                       'speechiness',
                                       'tempo'],
                           outputCol='features')

# Transform the data
train_df = assembler.transform(train_df)
test_df = assembler.transform(test_df)

knn = RandomForestClassifier(featuresCol='features', labelCol='genre',
                             maxDepth=5, maxBins=32, numTrees=10,
                             impurity='gini', subsamplingRate=0.8)

dt = DecisionTreeClassifier(featuresCol='features', labelCol='genre',
                            impurity='gini', maxDepth=5, maxBins=32)

rf = RandomForestClassifier(featuresCol='features', labelCol='genre',
                            impurity='gini', maxDepth=5, maxBins=32,
                            numTrees=10, subsamplingRate=0.8)

lr = LogisticRegression(featuresCol='features', labelCol='genre')

knnModel = knn.fit(train_df)
dtModel = dt.fit(train_df)
rfModel = rf.fit(train_df)
lrModel = lr.fit(train_df)

### Predict and Evaluate the models

In [30]:
# Make predictions on the test data
predictions_knn = knnModel.transform(test_df)
predictions_dt = dtModel.transform(test_df)
predictions_rf = rfModel.transform(test_df)
predictions_lr = lrModel.transform(test_df)

# Evaluate the accuracy of the models
evaluator = MulticlassClassificationEvaluator(labelCol="genre",
                                              predictionCol="prediction",
                                              metricName="accuracy")
accuracy_knn = evaluator.evaluate(predictions_knn)
accuracy_dt = evaluator.evaluate(predictions_dt)
accuracy_rf = evaluator.evaluate(predictions_rf)
accuracy_lr = evaluator.evaluate(predictions_lr)

print("k-Nearest Neighbors Accuracy:", accuracy_knn)
print("Decision Tree Accuracy:", accuracy_dt)
print("Random Forest Accuracy:", accuracy_rf)
print("Logistic Regression Accuracy:", accuracy_lr)

k-Nearest Neighbors Accuracy: 0.4104183109707972
Decision Tree Accuracy: 0.4059984214680347
Random Forest Accuracy: 0.4104183109707972
Logistic Regression Accuracy: 0.4320441988950276


Best Model is Logistic Regression with the highest accuracy = 0.4320441988950276